In [1]:
import pandas as pd
import re
from urllib.parse import urlparse

In [15]:
# Read the Sales Data file
df_BoxOffice = pd.read_csv('C:\\Users\\pc\\Downloads\\Test\\BoxOfficeWithUniqueID.csv',  encoding='ISO-8859-1') 
df_MovieIDTable = pd.read_csv('allMoviesWithUniqueID.csv', sep=None, engine='python', encoding='ISO-8859-1')
df_MovieIDTable = df_MovieIDTable.rename(columns={"ID": "Movie_ID"})

In [16]:
print(df_BoxOffice.columns)
print(df_MovieIDTable.columns)


Index(['Movie_ID', 'worldwide_box_office'], dtype='object')
Index(['TITLE', 'Movie_ID'], dtype='object')


In [18]:
# Merge the two dataframes on 'Movie_ID' column
df_NewMovieTable = pd.merge(df_MovieIDTable, df_BoxOffice[['Movie_ID', 'worldwide_box_office']], on='Movie_ID', how='left')

# Display the merged dataframe
df_NewMovieTable.head()


,TITLE,Movie_ID,worldwide_box_office
0,bakha satang,1,76576.0
1,antitrust,2,17865209.0
2,santitos,3,NaN
3,frank mcklusky c i,4,NaN
4,a walk to remember,5,46060861.0


In [20]:
#df_NewMovieTable.to_csv('NewMovieTable.csv', index=False, encoding='ISO-8859-1')

In [30]:
import psycopg2

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname='DMGroupAssignment',
        user='postgres',
        password='Okpanachi',
        host='localhost',
        port='5432'
    )

    cursor = conn.cursor()

    # Create the Movie table
    create_table_query = """
    CREATE TABLE IF NOT EXISTS MovieTable (
        Movie_ID INTEGER PRIMARY KEY,
        TITLE VARCHAR(255),
        worldwide_box_office DECIMAL(15, 2)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    # Insert data into the MovieTable
    insert_query = """
    INSERT INTO MovieTable (Movie_ID, TITLE, worldwide_box_office)
    VALUES (%s, %s, %s)
    ON CONFLICT (Movie_ID) DO UPDATE
    SET TITLE = EXCLUDED.TITLE,
        worldwide_box_office = EXCLUDED.worldwide_box_office;
    """

    # Loop through the DataFrame and execute the insert query for each row
    for index, row in df_NewMovieTable.iterrows():
        cursor.execute(insert_query, (row['Movie_ID'], row['TITLE'], row['worldwide_box_office']))

    # Commit the transaction to save changes to the database
    conn.commit()

except Exception as error:
    print(f"Error: {error}")

finally:
    # Ensure the cursor and connection are closed
    if cursor:
        cursor.close()
    if conn:
        conn.close()
